In [315]:
#!g1.1
#%pip install opencv-python editdistance ml_collections wandb albumentations
#%pip install bezier
#%pip install typing_extensions --upgrade
#%pip install pandas -U
%pip install numpy==1.21

Execute error: Servant g1.1 not allocated: Internal Error

In [507]:
#!g1.1
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF

import torchvision as tv

import numpy as np

import typing as tp
import os

import cv2

import matplotlib.pyplot as plt
import pandas as pd
import math
import itertools

from editdistance import eval as edit_distance

from tqdm import tqdm

import errno

from ml_collections import ConfigDict

import importlib
import wandb

import albumentations as A

import inspect

In [508]:
#!g1.1
from diploma_code.patched_hwb import HandWrittenBlot

# from diploma_code.model import (
#     ConvBlock, FusedInvertedBottleneck, ReduceBlock, Backbone, PositionalEncoding,
#     TransformerEncoder, CTCRawDecoder, CTCDecoderModel, ParallelModel, make_single_model, make_model
# )

from diploma_code.model_v2 import (
    Resnet34Backbone, BiLSTMEncoder, PositionalEncoding, CoolTransformerEncoderLayer, TransformerEncoder, 
    CTCDecoderModel, ParallelModel, make_single_model_v2, make_model_v2
)
from diploma_code.transforms_torch import (
    VerticalRandomMasking, HorizontalResizeOnly, GaussianNoise, 
    RandomHorizontalStretch, RandomChoiceN
)
from diploma_code.char_encoder import (
    CharEncoder
)
# from diploma_code.data_loader.mjsynth import (
#     load_mjsynth_chars, load_mjsynth_samples
# )
from diploma_code.dataset import (
    BaseLTRDataset, LongLinesLTRDataset
)
from diploma_code.make_loader import (
    make_char_encoder, make_datasets, make_dataloaders
)
from diploma_code.optimizing import (
    pytorch_make_optimizer, StepLRWithWarmup, ReverseSqrtWithLinearWarmup, 
    ConstantLambdaLR, make_lr_scheduler_torch, make_lr_scheduler_lambda
)
from diploma_code.utils import (
    log_metric_wandb, batch_to_device, seed_everything
)
from diploma_code.evaluation import (
    my_ctc_loss, my_dml_loss, my_mcl_loss, my_dml_loss_tr, 
    trkl_div, decode_ocr_probs, get_edit_distance, 
    EpochValueProcessor, EpochDMLProcessor, CERProcessor
)
from diploma_code.trainer import (
    LTRTrainer
)

from diploma_code.config import default_diploma_config, get_config

from diploma_code.configs import BaseDatasetConfig
from diploma_code.configs import IamConfig
from diploma_code.configs import SaintgallConfig



In [509]:
#!g1.1
import diploma_code
import diploma_code.data_loader

importlib.reload(diploma_code.char_encoder)

importlib.reload(diploma_code.config)
importlib.reload(diploma_code.dataset)

importlib.reload(diploma_code.data_loader)
importlib.reload(diploma_code.evaluation)

importlib.reload(diploma_code.optimizing)

importlib.reload(diploma_code.model_v2)
importlib.reload(diploma_code.utils)
importlib.reload(diploma_code.trainer)

importlib.reload(diploma_code.make_loader)
importlib.reload(diploma_code.make_transforms)

importlib.reload(diploma_code.transforms_functional)

importlib.reload(diploma_code.configs)





<module 'diploma_code.configs' from '/home/jupyter/work/resources/mutual_htr/mutual_htr/diploma_code/configs/__init__.py'>

In [510]:
#!g1.1
wandb.login()

wandb: Currently logged in as: kafka_zhuk. Use `wandb login --relogin` to force relogin


True

In [511]:
#!g1.1
cfg = get_config()
cfg.training.eval_epochs_interval = 3
cfg.training.eval_test_interval = 3
cfg.training.snapshot_epochs_interval = 3
cfg.data.root_path = '/home/jupyter/mnt/datasets/diploma/'
cfg.data.iam.path = '/home/jupyter/mnt/datasets/diploma/'
cfg.data.saintgall.path = '/home/jupyter/mnt/datasets/diploma'

cfg.data.dataset = 'iam'

cfg.model.type = 'single'
# cfg.model.first.backbone.Resnet34Backbone.pretrained = True
# cfg.model.first.encoder.constructor = 'TransformerEncoder'
# cfg.model.first.encoder.TransformerEncoder.num_layers = 4
# cfg.model.first.encoder.TransformerEncoder.num_heads = 4
# cfg.model.first.encoder.TransformerEncoder.hidden_features = 256

# cfg.model.second.backbone.Resnet34Backbone.pretrained = True
# cfg.model.second.encoder.constructor = 'TransformerEncoder'
# cfg.model.second.encoder.TransformerEncoder.num_layers = 4
# cfg.model.second.encoder.TransformerEncoder.num_heads = 4
# cfg.model.second.encoder.TransformerEncoder.hidden_features = 256

cfg.device = 'cuda:0'

cfg.training.batch_size = 16
cfg.training.grad_clip_value = 1
cfg.training.num_epochs = 100

# cfg.lr_scheduler.use_lambda = True
# cfg.lr_scheduler.constructor = "ReverseSqrtWithLinearWarmup"
# cfg.lr_scheduler.params = ConfigDict()
# cfg.lr_scheduler.params.warmup_steps = 3000

cfg.lr_scheduler.params.epochs = 100
cfg.lr_scheduler.params.max_lr = 1e-3
cfg.lr_scheduler.params.steps_per_epoch = (cfg.data.get_ref(cfg.data.dataset).get().get_ref('length') + cfg.training.get_ref('batch_size') - 1) // cfg.training.get_ref('batch_size')

cfg.backbone_lr_scheduler.params.epochs = 100
cfg.backbone_lr_scheduler.params.max_lr = 1e-3
cfg.backbone_lr_scheduler.params.steps_per_epoch = (cfg.data.get_ref(cfg.data.dataset).get().get_ref('length') + cfg.training.get_ref('batch_size') - 1) // cfg.training.get_ref('batch_size')

cfg.optimizer.weight_decay = 0.01
#cfg.optimizer.params.lr = 1e-5

cfg.backbone_optimizer.weight_decay = 0.01
#cfg.backbone_optimizer.params.lr = 1e-5

#cfg.training.checkpoints_folder = './checkpoints_single_lstm_67_60ep_sg'
cfg.training.checkpoints_folder = './checkpoints_single_lstm_42_200_epoch'
cfg.load_checkpoints_folder = './checkpoints_single_lstm_42_200_epoch'

cfg.training.load_from_checkpoint = True

cfg.eval = True
#cfg.seed = 42
cfg.seed = 67
#cfg.seed = 95

cfg.data.saintgall.chars = ' !"#&\'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

# 3 epoch warmup

#cfg.dml.warmup_steps = 3 * (cfg.data.get_ref(cfg.data.dataset).get().get_ref('length') + cfg.training.get_ref('batch_size') - 1) // cfg.training.get_ref('batch_size')
cfg.dml.warmup_steps = 0

In [512]:
#!g1.1
cfg

backbone_lr_scheduler:
  constructor: torch.optim.lr_scheduler.OneCycleLR
  params:
    anneal_strategy: cos
    epochs: !!python/object:ml_collections.config_dict.config_dict.FieldReference
      _field_type: &id001 !!python/name:builtins.int ''
      _ops: []
      _required: false
      _value: 100
    final_div_factor: 100000
    max_lr: 0.001
    pct_start: 0.1
    steps_per_epoch: !!python/object:ml_collections.config_dict.config_dict.FieldReference
      _field_type: *id001
      _ops: []
      _required: false
      _value: 604
  use_lambda: false
backbone_optimizer:
  constructor: torch.optim.AdamW
  params:
    betas: &id008 !!python/tuple
    - 0.9
    - 0.999
    lr: 0.0002
  weight_decay: 0.01
data:
  dataset: iam
  iam: !!python/object:ml_collections.config_dict.config_dict.FieldReference
    _field_type: !!python/name:builtins.dict ''
    _ops: []
    _required: false
    _value:
      blank: "\u03B2"
      chars: ' !"#&''()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZab

In [513]:
#!g1.1
os.makedirs(cfg.training.checkpoints_folder, exist_ok=True)

In [514]:
#!g1.1
#!rm checkpoints_lstm_67_merged_10/model.pth

In [515]:
#!g1.1
seed_everything(cfg.seed)

In [493]:
#!g1.1
#LTRTrainer.merge_model_checkpoints('./checkpoints_single_lstm_67_explore_10', './checkpoints_single_lstm_67', './checkpoints_lstm_67_merged_10')

In [491]:
#!g1.1
!ls checkpoints_lstm_67_merged_10

train_state.pth


In [516]:
#!g1.1
ltr = LTRTrainer(cfg)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /tmp/xdg_cache/torch/hub/checkpoints/resnet34-b627a593.pth


In [451]:
#!g1.1
#ltr.validate(mode='valid')

In [502]:
#!g1.1
ltr.train('lstm_saintgall_6_54_dml_notr_2')


wandb: wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.2
wandb: Run data is saved locally in /home/jupyter/work/resources/mutual_htr/mutual_htr/wandb/run-20230622_150111-votx4nvj
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run lstm_saintgall_6_54_dml_notr_2
wandb: ⭐️ View project at https://wandb.ai/kafka_zhuk/diploma_dml
wandb: 🚀 View run at https://wandb.ai/kafka_zhuk/diploma_dml/runs/votx4nvj
100%|██████████| 45/45 [00:14<00:00,  3.11it/s]
wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb: backbone_grad_norm/train ▂▂▂▂▁▁▁▁█▂▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
wandb:      backbone_lr_0/train ▁▃▆▇██████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
wandb:      backbone_lr_1/train ▁▃▆▇██████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
wandb:         epoch_cer_1/test █▅▄▄▂▃▃▂▂▂▂▁▁▁▁▁▁▁
wandb:        epoch_cer_1/valid █▅▄▃▃▃▃▃▂▁▁▂▁▁▁▁▁▁
wandb:         epoch_cer_2/test █▅▄▃

In [ ]:
#!g1.1
wandb.finish()

In [ ]:
#!g1.1
